# Tageszeiten Analyse - (15min Werte)

In [93]:
import pandas as pd
import altair as alt

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [94]:
df = pd.read_csv("data/rad_2022_15min_06_06_23_r.csv", low_memory=False)


In [95]:
df.head()

,datum,uhrzeit_start,uhrzeit_ende,zaehlstelle,richtung_1,richtung_2,gesamt,kommentar
0,2022-01-01,00:00:00,00:15:00,Arnulf,0.0,0.0,0.0,NaN
1,2022-01-01,00:15:00,00:30:00,Arnulf,0.0,0.0,0.0,NaN
2,2022-01-01,00:30:00,00:45:00,Arnulf,0.0,0.0,0.0,NaN
3,2022-01-01,00:45:00,01:00:00,Arnulf,1.0,0.0,1.0,NaN
4,2022-01-01,01:00:00,01:15:00,Arnulf,1.0,0.0,1.0,NaN


In [96]:
df.dtypes

datum             object
uhrzeit_start     object
uhrzeit_ende      object
zaehlstelle       object
richtung_1       float64
richtung_2       float64
gesamt           float64
kommentar         object
dtype: object

In [99]:
# Aktuelle Pfade
file_paths = [
     "data/rad_2022_tage_19_06_23_r.csv",
     "data/rad_2021_tage_19_06_23_r.csv",
     "data/rad_2020_tage_19_06_23_r.csv",
     "data/rad_2019_tage_19_06_23_r.csv",
     "data/rad_2018_tage_19_06_23_r.csv",
     "data/rad_2017_tage_19_06_23_r.csv",
     "data/rad_2016_tage_19_06_23_r.csv",
     "data/rad_2015_tage_19_06_23_r.csv",
     "data/rad_2014_tage_19_06_23_r.csv",
     "data/rad_2013_tage_19_06_23_r.csv",
     "data/rad_2012_tage_19_06_23_r.csv"
]

In [100]:
df_list = [
    
]

for file_path in file_paths:
    df_temp = pd.read_csv(file_path)
    df_temp['wochentag'] = df['datum'].dt.day_name(locale='de_DE')
    df_list.append(df_temp[['wochentag', 'zaehlstelle', 'gesamt']])
    

# Einzelne DataFrames zu einem Gesamt-DataFrame zusammenführen
df_combined = pd.concat(df_list, ignore_index=True)


In [101]:
# Spalte 'datum' in datetime umwandeln
df['datum'] = pd.to_datetime(df['datum'])

# Wochentage hinzufügen
df['wochentag'] = df['datum'].dt.day_name(locale='de_DE')

# Neues DataFrame anzeigen
print(df)

            datum uhrzeit_start uhrzeit_ende zaehlstelle  richtung_1  \
0      2022-01-01      00:00:00     00:15:00      Arnulf         0.0   
1      2022-01-01      00:15:00     00:30:00      Arnulf         0.0   
2      2022-01-01      00:30:00     00:45:00      Arnulf         0.0   
3      2022-01-01      00:45:00     01:00:00      Arnulf         1.0   
4      2022-01-01      01:00:00     01:15:00      Arnulf         1.0   
...           ...           ...          ...         ...         ...   
210235 2022-12-31      22:45:00     23:00:00     Erhardt         4.0   
210236 2022-12-31      23:00:00     23:15:00     Erhardt         4.0   
210237 2022-12-31      23:15:00     23:30:00     Erhardt         5.0   
210238 2022-12-31      23:30:00     23:45:00     Erhardt         2.0   
210239 2022-12-31      23:45:00     00:00:00     Erhardt         5.0   

        richtung_2  gesamt kommentar wochentag  
0              0.0     0.0       NaN   Samstag  
1              0.0     0.0       NaN 

In [102]:
# Reihenfolge der Wochentage festlegen
wochentage_order = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']

# Altair-Diagramm erstellen
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('wochentag:N', title='Wochentag', sort=wochentage_order).stack('zero'),
    y=alt.Y('sum(gesamt):Q', title='Gesamt'),
    color=alt.Color('zaehlstelle'),
    tooltip=[alt.Tooltip('wochentag:N', title='Wochentag'), alt.Tooltip('sum(gesamt):Q', title='Anzahl Radfahrer')],
).properties(
    width=600,
    height=400,
    title='Anzahl Radfahrer pro Wochentag in 2022'
)



chart 


alt.Chart(...)

In [103]:


# Spalte 'datum' in datetime umwandeln
df['datum'] = pd.to_datetime(df['datum'])

# Wochentage hinzufügen
df['wochentag'] = df['datum'].dt.day_name(locale='de_DE')  # Deutsch

# Reihenfolge der Wochentage festlegen
wochentage_order = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']

# Daten vorbereiten (Gruppieren und Aggregieren)
df_grouped = df.groupby(['zaehlstelle', 'wochentag']).agg({'gesamt': 'sum'}).reset_index()

# Altair-Diagramm erstellen
chart = alt.Chart(df_grouped, title="Anzahl Radfahrer pro Wochentag und Zaehlstelle").mark_rect().encode(
    x=alt.X('wochentag:N', title='Wochentag', sort=wochentage_order),
    y=alt.Y('zaehlstelle:N', title='Zaehlstelle'),
    color=alt.Color('gesamt:Q', title='Gesamt', scale=alt.Scale(scheme='blues')),
    tooltip=[
        alt.Tooltip('wochentag:N', title='Wochentag'),
        alt.Tooltip('zaehlstelle:N', title='Zählstelle'),
        alt.Tooltip('gesamt:Q', title='Gesamt'),
    ],
).properties(
    width=600,
    height=400
)

# Configure text
text = chart.mark_text(baseline='middle').encode(
    alt.Text('gesamt:Q')
)

# Configure text
text = base.mark_text(baseline='middle').encode(
    alt.Text('mean_horsepower:Q', format=".0f"),
    color=alt.condition(
        alt.datum.mean_horsepower > 150,
        alt.value('black'),
        alt.value('white')
    )
)

# Diagramm anzeigen
chart + text


alt.LayerChart(...)

In [106]:
# Configure common options. We specify the aggregation
# as a transform here so we can reuse it in both layers.
base = alt.Chart(df).transform_aggregate(
    gesamt='gesamt:Q',
    groupby=['wochentag', 'zaehlstelle']
).encode(
    alt.X('wochentag:N'),
    alt.Y('zaehlstelle:N'),
)

# Configure heatmap
heatmap = base.mark_rect().encode(
    alt.Color('gesamt:Q')
        .scale(scheme='viridis')
        .title("Anzahl Radfahrer pro Wochentag und Zaehlstelle")
)

# Configure text
text = base.mark_text(baseline='middle').encode(
    alt.Text('gesamt:Q', format=".0f"),
    color=alt.condition(
        alt.datum.gesamt > 150,
        alt.value('black'),
        alt.value('white')
    )
)

# Draw the chart
heatmap + text

SchemaValidationError: '{'field': 'gesamt', 'as': 'gesamt'}' is an invalid value.

'op' is a required property